In [1]:
# Load data - run prep_exp.ipynb first
import pickle

pickle_in = open("temp.pkl","rb")
data = pickle.load(pickle_in)

# # If it's not there, do the following:
# import sys
# import os
# import numpy as np
# from str2bool import str2bool
# from rpy2.robjects.packages import STAP
# import rpy2.robjects as robjects
# from rpy2.robjects import pandas2ri, numpy2ri
# from rpy2.robjects.lib.dplyr import DataFrame
# from rpy2.robjects.packages import importr

# pandas2ri.activate()
# numpy2ri.activate()

# # Some temporary arguments for testing
# include_lab = str2bool("T")  # Include lab features?
# include_ethdon = str2bool("T")  # Include ethnicity + donor details?
# lag = int("1")  # Number of lag variables
# eq_train_ratio = str2bool("T")  # Train on equal case:control ratio?
# __file__ = '/h/angeliney/projects/SRTR/prep_exp.py'
# visit_type = "first"
# output = "temp"
# post2000 = True

# # Get features based on these inputs
# with open(os.path.join(os.path.dirname(__file__), 'features.R'), 'r') as f:
#     string = f.read()
# features_file = STAP(string, "features")
# features_to_use = features_file.features.rx2("clin")
# if include_lab:
#     features_to_use = features_to_use + features_file.features.rx2("lab")

# if include_ethdon:
#     features_to_use = features_to_use + features_file.features.rx2("eth") + features_file.features.rx2("don")

# timedep_cols = np.intersect1d(features_to_use, features_file.timedep_features)
# cov_cols = np.setdiff1d(features_to_use, timedep_cols)

# if eq_train_ratio:
#     eq_cases_train_cols = np.array(["TRR_ID", "is_diab"])
# else:
#     eq_cases_train_cols = np.array()
    
# # Read RDS files (load data table)
# readRDS = robjects.r['readRDS']
# tx_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'tx_li_formatted.rds'))
# txf_li_study = readRDS(os.path.join(os.path.dirname(__file__), 'txf_li_formatted.rds'))

# # Merge them
# with open(os.path.join(os.path.dirname(__file__), 'functions.R'), 'r') as f:
#     string = f.read()
# functions = STAP(string, "functions")

# merged = functions.combine_tx_txf(tx_li_study, txf_li_study, np.setdiff1d(cov_cols, "age"), timedep_cols, lag)

# df = pandas2ri.ri2py_dataframe(DataFrame(merged).filter('time_next_followup > time_since_transplant'))

# #Prep data for model training
# cols = np.concatenate((timedep_cols, cov_cols))
# if lag > 0:
#     for l in range(1,  lag + 1):
#         cols = np.append(cols, list(map(lambda x: '{}_{}'.format(x, l), timedep_cols)))

# subset_cols = np.concatenate((['transplant_year', 'TRR_ID', 'age'], cols, ['is_diab', 'time_since_transplant',
#                                                                    'time_next_followup', 'time_to_diab',
#                                                                    'diab_time_since_tx', 'diab_in_1_year',
#                                                                    'diab_now']))
# df = df.dropna(subset=subset_cols)
# df_test = df[(df.transplant_year.astype(int) >= 2011) & (df.time_to_diab >= 0)]
# df_nontest = df[(df.transplant_year.astype(int) < 2011) & (df.time_to_diab >= 0)]

# num_folds = 5
# nontest_y = df_nontest.drop_duplicates(subset=['TRR_ID', 'is_diab']).is_diab
# caret = importr('caret')
# folds = caret.createFolds(nontest_y, num_folds, False)

# data = {'test': df_test, 'train': df_nontest, 'cols': cols, 'eq_cases_train_cols': eq_cases_train_cols,
#             'folds': folds}

# import pickle

# # Save data in case kernel got restarted
# pickle.dump(data, open("temp.pkl", "wb")) 

In [7]:
import argparse
import sys
#import Bayesian_Optimization as BayesOpt
import os
import scipy.io as sio
from survivalnet.optimization import SurvivalAnalysis
import numpy as np
from survivalnet.train import train
import theano
import cPickle
import pandas

In [14]:
# Some settings
visit_type="random"
if visit_type == "last":
    time_col = "time_since_transplant"
    cols = data['cols']
else:
    time_col = "time_to_diab"
    cols = np.append(data['cols'], 'time_since_transplant')
    
eq_train_ratio = True

# Subset train, val, test
nontest_ids = data['train'].drop_duplicates(subset=['TRR_ID', 'is_diab']).TRR_ID
i=1
train_ids = nontest_ids[np.array(data['folds']) != i]
val_ids = np.setdiff1d(nontest_ids, train_ids)

train = data['train'][data['train'].TRR_ID.isin(train_ids)]
train = train[train.is_diab == 1]  # Only include censored objects in training. Otherwise, comment out
val = data['train'][data['train'].TRR_ID.isin(val_ids)] 
val = val[val.is_diab == 1]  # Only include censored objects in training. Otherwise, comment out
test = data['test']

from functions import equalize_num_case_control, filter_train_by_visit
train = filter_train_by_visit(visit_type, data['train'])
train = equalize_num_case_control(train, data['eq_cases_train_cols'])

In [31]:
do_bayes_opt = False
epochs = 10
output_path = "results"
opt = 'GDLS'
pretrain_config = None

# The results in the paper are averaged over 20 random assignment of samples
# to training/validation/testing sets.
cindex_results_train =[]
cindex_results =[]

# Caclulates the risk group for every patient i: patients whose time of
# death is greater than that of patient i.
sa = SurvivalAnalysis()
train_set = {}
val_set = {}
test_set = {}
train_set['X'], train_set['T'], train_set['O'], train_set['A'] = sa.calc_at_risk(
    train[cols].values.astype("float32"),
    train[time_col].values.astype("float32"),
    train.is_diab.values.astype("int32"))

val_set['X'], val_set['T'], val_set['O'], val_set['A'] = sa.calc_at_risk(
    val[cols].values.astype("float32"),
    val[time_col].values.astype("float32"),
    val.is_diab.values.astype("int32"))

test_set['X'], test_set['T'], test_set['O'], test_set['A'] = sa.calc_at_risk(
    test[cols].values.astype("float32"),
    test[time_col].values.astype("float32"),
    test.is_diab.values.astype("int32"))

pretrain_set = train_set['X']


n_layers = 1
n_hidden = 100
do_rate = 0.5
lambda1 = 0
lambda2 = 0
nonlin = np.tanh # or nonlin = theano.tensor.nnet.relu

# Prints experiment identifier.
expID = 'nl{}-hs{}-dor{}_nonlin{}_id{}'.format(str(n_layers), str(n_hidden), str(do_rate), str(nonlin), str(i))

finetune_config = {'ft_lr': 0.0001, 'ft_epochs': epochs}

#_, train_cindices, _, test_cindices, _, _, model, _ = train(pretrain_set, train_set, test_set, pretrain_config, 
#                                                            finetune_config, n_layers, n_hidden, 
#                                                            dropout_rate=do_rate, lambda1=lambda1, lambda2=lambda2, 
#                                                            non_lin=nonlin, optim=opt, verbose=True, earlystp=False)

#print(train_cindices[np.argmax(test_cindices)])
#print(max(test_cindices))

In [36]:
verbose = True
earlystp = False
dropout_rate = do_rate
non_lin = nonlin
optim = opt

import numpy
import os
import sys
import theano
import timeit

from survivalnet.model import Model
from survivalnet.optimization import BFGS, GDLS, SurvivalAnalysis, isOverfitting

In [38]:
LEARNING_RATE_DECAY = 1 
finetune_lr = theano.shared(numpy.asarray(finetune_config['ft_lr'], dtype=theano.config.floatX))

numpy_rng = numpy.random.RandomState(1111)

# Construct the stacked denoising autoencoder and the corresponding
# supervised survival network.
model = Model(
        numpy_rng = numpy_rng,
        n_ins = train_set['X'].shape[1],
        hidden_layers_sizes = [n_hidden] * n_layers,
        n_outs = 1,
        dropout_rate=dropout_rate,
        lambda1 = lambda1,
        lambda2 = lambda2,
        non_lin=non_lin)

In [42]:
########################
# FINETUNING THE MODEL #
########################
test, train = model.build_finetune_functions(learning_rate=finetune_lr)

train_cindices = []
test_cindices = []
train_costs = []
test_costs = []

if optim == 'BFGS':        
    bfgs = BFGS(model, train_set['X'], train_set['O'], train_set['A'])
elif optim == 'GDLS':
    gdls = GDLS(model, train_set['X'], train_set['O'], train_set['A'])
survivalAnalysis = SurvivalAnalysis()    

# Starts the training routine.
for epoch in range(finetune_config['ft_epochs']):

    # Creates masks for dropout during training.
    train_masks = [
        numpy_rng.binomial(n=1, p=1-dropout_rate, 
                           size=(train_set['X'].shape[0], n_hidden))
        for i in range(n_layers)]

    # Creates dummy masks for testing.
    test_masks = [
        numpy.ones((test_set['X'].shape[0], n_hidden), dtype='int64')
        for i in range(n_layers)]

    # BFGS() and GDLS() update the gradients, so we only serve (test) the
    # model to calculate cost, risk, and cindex on training set.
    if optim == 'BFGS':        
        bfgs.BFGS(train_masks)
        train_cost, train_risk, train_features = test(
            train_set['X'], train_set['O'], train_set['A'], 1, *train_masks)
    elif optim == 'GDLS':        
        gdls.GDLS(train_masks)
        train_cost, train_risk, train_features = test(
            train_set['X'], train_set['O'], train_set['A'], 1, *train_masks)
    # In case of GD, uses the train function to update the gradients and get
    # training cost, risk, and cindex at the same time.
    elif optim == 'GD':
        train_cost, train_risk, train_features = train(
            train_set['X'], train_set['O'], train_set['A'], 1, *train_masks)
    train_ci = survivalAnalysis.c_index(train_risk, train_set['T'], 1 - train_set['O'])

    # Calculates testing cost, risk and cindex using th eupdated model.
    test_cost, test_risk, _ = test(test_set['X'], test_set['O'], test_set['A'], 0, *test_masks)
    test_ci = survivalAnalysis.c_index(test_risk, test_set['T'], 1 - test_set['O'])

    train_cindices.append(train_ci)
    test_cindices.append(test_ci)

    train_costs.append(train_cost)
    test_costs.append(test_cost)
    if verbose: 
        print (('epoch = {}, trn_cost = {}, trn_ci = {}, tst_cost = {},'
                ' tst_ci = {}').format(epoch, train_cost, train_ci,
                                       test_cost, test_ci))
    if earlystp and epoch >= 15 and (epoch % 5 == 0):
        if verbose:
            print 'Checking overfitting!'
        check, max_iter = isOverfitting(numpy.asarray(test_cindices))
        if check:                
            print(('Training Stopped Due to Overfitting! cindex = {},'
                   ' MaxIter = {}').format(test_cindices[max_iter], max_iter))
            break
    else: max_iter = epoch
    sys.stdout.flush()
    decay_learning_rate = theano.function(
            inputs=[], outputs=finetune_lr,
            updates={finetune_lr: finetune_lr * LEARNING_RATE_DECAY})    
    decay_learning_rate()
    epoch += 1
    if numpy.isnan(test_cost): break 
if verbose: 
    print 'C-index score after {} epochs is: {}'.format(max_iter, max(test_cindices))

KeyboardInterrupt: 

In [43]:
test_ci = survivalAnalysis.c_index(test_risk, test_set['T'], 1 - test_set['O'])

train_cindices.append(train_ci)
test_cindices.append(test_ci)

train_costs.append(train_cost)
test_costs.append(test_cost)
if verbose: 
    print (('epoch = {}, trn_cost = {}, trn_ci = {}, tst_cost = {},'
            ' tst_ci = {}').format(epoch, train_cost, train_ci,
                                   test_cost, test_ci))

epoch = 0, trn_cost = -54089.9651783, trn_ci = 0.497985618024, tst_cost = -68140.4049368, tst_ci = 0.471169146678


In [ ]:
with file(os.path.join(output_path, 'final_model'), 'wb') as f:
    cPickle.dump(model, f, protocol=cPickle.HIGHEST_PROTOCOL)

outputFileName = os.path.join(output_path, 'c_index_list.csv')
df = pandas.DataFrame(test_cindices)
df.to_csv(outputFileName)